# 多层网络模型

本小节主要介绍多层神经网络模型的设计，使用线性回归实例作为讲解实例。


本文将使用MIndspore构建线性函数来实现多层神经网络模型的设计

定义多层神经网络，内部有两层全连接层组成的网络，包含五个入参和一个出参

In [1]:

import numpy as np

import mindspore.nn as nn
import mindspore.ops as ops
import mindspore.dataset as ds
from mindspore import ParameterTuple

class LinearNet(nn.Cell):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.dense1 = nn.Dense(5, 32)
        self.dense2 = nn.Dense(32, 1)

    def construct(self, x):
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        return x


产生输入数据

In [2]:
np.random.seed(4)

class DatasetGenerator:
    def __init__(self):
        self.data = np.random.randn(5, 5).astype(np.float32)
        self.label = np.random.randn(5, 1).astype(np.int32)

    def __getitem__(self, index):
        return self.data[index], self.label[index]

    def __len__(self):
        return len(self.data)

数据处理

In [3]:
# 对输入数据进行处理
dataset_generator = DatasetGenerator()
dataset = ds.GeneratorDataset(dataset_generator, ["data", "label"], shuffle=True)
dataset = dataset.batch(32)

# 实例化网络
net = LinearNet()

In [4]:
class TrainOneStepCell(nn.Cell):
    def __init__(self, network, optimizer, sens=1.0):
        """参数初始化"""
        super(TrainOneStepCell, self).__init__(auto_prefix=False)
        self.network = network
        # 使用tuple包装weight
        self.weights = ParameterTuple(network.trainable_params())
        self.optimizer = optimizer
        # 定义梯度函数
        self.grad = ops.GradOperation(get_by_list=True, sens_param=True)
        self.sens = sens

    def construct(self, data, label):
        """构建训练过程"""
        weights = self.weights
        loss = self.network(data, label)
        # 为反向传播设定系数
        sens = ops.Fill()(ops.DType()(loss), ops.Shape()(loss), self.sens)
        grads = self.grad(self.network, weights)(data, label, sens)
        return loss, self.optimizer(grads)

网络训练

In [5]:
# 设定损失函数
crit = nn.MSELoss()
# 设定优化器
opt = nn.Adam(params=net.trainable_params())
# 引入损失函数
net_with_criterion = nn.WithLossCell(net, crit)
# 自定义网络训练
train_net = TrainOneStepCell(net_with_criterion, opt)

# 获取训练过程数据
for d in dataset.create_dict_iterator():
    for i in range(300):
        train_net(d["data"], d["label"])
        print(net_with_criterion(d["data"], d["label"]))

0.7990034
0.7983502
0.79763794
0.7968837
0.79607975
0.7951586
0.7941489
0.79305816
0.79186016
0.7905208
0.7890532
0.7874637
0.7857695
0.7839347
0.7819258
0.7797663
0.7774768
0.7750362
0.77243507
0.7697221
0.7668141
0.76372147
0.76045555
0.7570098
0.75342995
0.7496825
0.7457578
0.74163413
0.73731923
0.7328435
0.7281942
0.7234098
0.71840584
0.7132035
0.7078464
0.702299
0.69656545
0.6906762
0.6846032
0.6783771
0.67199236
0.6654457
0.6587184
0.65185827
0.64482105
0.63766426
0.6303346
0.6228629
0.6152377
0.6075232
0.5996808
0.5917364
0.58367527
0.575485
0.5672281
0.55884343
0.5503585
0.54185987
0.5333041
0.52458435
0.5159018
0.5071903
0.4983678
0.48953933
0.4806924
0.47186017
0.46301675
0.45415074
0.44533083
0.43652833
0.42769396
0.41888797
0.41016698
0.4015411
0.3929267
0.38435754
0.37589186
0.36748075
0.35914958
0.35091537
0.34274825
0.3347097
0.3267454
0.31893194
0.3112413
0.30364633
0.29615802
0.28881368
0.28157693
0.27447253
0.26750478
0.26064822
0.25397402
0.2473329
0.24087876
0.23456